<a href="https://colab.research.google.com/github/ALisichka009/PUZANOVA-2303-Lab1/blob/Lab2/SQL-%D0%B8%D0%BD%D1%8A%D0%B5%D0%BA%D1%86%D0%B8%D0%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import hashlib
import re
from cryptography.fernet import Fernet

In [66]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [76]:
file_path = "/content/drive/MyDrive/Laptop_price=.csv"
try:
  df = pd.read_csv(file_path)
except Exception as e:
  print (f"ошибка загрузки файла: {e}")

In [ ]:
# Проверка CSV на уязвимости и добавление проверки на пробелы
def check_csv_injection (df):
  dangerous_chars = ('=', '+', '-', '@', ' ')
  for col in df.select_dtypes(include=['object']).columns:
    if df[col].astype(str).apply(lambda x: x.startswith(dangerous_chars)).any():
        print (f"Обнаружены потенциальные CSV-инъекции в столбце {col}!")
    else:
        print (f"Столбец {col} безопасен.")

check_csv_injection(df)




Обнаружены потенциальные CSV-инъекции в столбце Brand!


In [82]:
# Фильтрация данных от SQL-инъекций и XSS-атак
def clean_input(value):
  sql_keywords = ["SELECT", "DROP", "DELETE", "INSERT", "UPDATE", "ALERT", "UNION"]
  xss_patterns = [r'<script.*?>.*?</script>', r'javascript:.*', r'onerror=.*']

   # Проверка на SQL-ключевые слова
  for keyword in sql_keywords:
    if keyword.lower() in value.lower():
      return "[BLOCKED]"

  # Проверка на комментарии SQL
    if any(char in value for char in ['--']):
        return "[BLOCKED]"

  return value

df = df.map(lambda x: clean_input(str(x)) if isinstance(x, str) else x)
print("Фильтрация данных завершена.")

Фильтрация данных завершена.


In [83]:
#Шифрование данных
key = Fernet.generate_key()
cipher = Fernet(key)

def encrypt_ram(ram):
  return cipher.encrypt(str(ram).encode()).decode()
df['ram_Encrypted'] = df['RAM_Size'].apply(encrypt_ram)
print("Столбец ram_Encrypted зашифрован")

#Дешифрование данных

def decrypt_ram(encrypted_ram, row_index):
  if row_index < 5:  # Дешифруем только первые 5 строк
        return cipher.decrypt(encrypted_ram.encode()).decode()
  else:
        return encrypted_ram  # Оставляем зашифрованным

df['ram_DEncrypted'] = df.reset_index().apply(
    lambda x: decrypt_ram(x['ram_Encrypted'], x['index']), axis=1)
print("Столбец ram_Encrypted Расшифрован")

output_path = "/content/drive/MyDrive/Laptop_desecured.csv"
df.to_csv(output_path, index=False)
print(f"Файл сохранен: {output_path}")

Столбец ram_Encrypted зашифрован
Столбец ram_Encrypted Расшифрован
Файл сохранен: /content/drive/MyDrive/Laptop_desecured.csv
